In [1]:
from __future__ import print_function

import os
from sklearn.model_selection import train_test_split
import nltk
import numpy as np
import string

with open('/Users/anseon-yeong/수업/2021-2/텍스트정보처리/week13/13주차_txt/alice_in_wonderland.txt', 'r') as content_file:
    content = content_file.read()


In [10]:
content2 = " ".join("".join([" " if ch in string.punctuation else ch for ch in content]).split())
 
tokens = nltk.word_tokenize(content2)
tokens = [word.lower() for word in tokens if len(word)>=2]

N = 3
quads = list(nltk.ngrams(tokens,N))

newl_app = []
for ln in quads:
    newl = " ".join(ln)        
    newl_app.append(newl)

In [11]:

#단어 벡터화
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

x_trigm = []
y_trigm = []

for l in newl_app:
    x_str = " ".join(l.split()[0:N-1])
    y_str = l.split()[N-1]   
    x_trigm.append(x_str)
    y_trigm.append(y_str)

x_trigm_check = vectorizer.fit_transform(x_trigm).todense()
y_trigm_check = vectorizer.fit_transform(y_trigm).todense()


In [12]:
#단어를 정수, 정수에서 단어로 변환하기 위한 딕셔너리 
dictnry = vectorizer.vocabulary_
rev_dictnry = {v:k for k,v in dictnry.items()}

X = np.array(x_trigm_check)
Y = np.array(y_trigm_check)

Xtrain, Xtest, Ytrain, Ytest,xtrain_tg,xtest_tg = train_test_split(X, Y,x_trigm, test_size=0.3,random_state=42)

print("X Train shape",Xtrain.shape, "Y Train shape" , Ytrain.shape)
print("X Test shape",Xtest.shape, "Y Test shape" , Ytest.shape)

X Train shape (17947, 2559) Y Train shape (17947, 2559)
X Test shape (7692, 2559) Y Test shape (7692, 2559)


In [13]:
# Model Building
from keras.layers import Input,Dense,Dropout
from keras.models import Model

np.random.seed(42)

BATCH_SIZE = 128
NUM_EPOCHS = 100

input_layer = Input(shape = (Xtrain.shape[1],),name="input")
first_layer = Dense(1000,activation='relu',name = "first")(input_layer)
first_dropout = Dropout(0.5,name="firstdout")(first_layer)

second_layer = Dense(800,activation='relu',name="second")(first_dropout)

third_layer = Dense(1000,activation='relu',name="third")(second_layer)
third_dropout = Dropout(0.5,name="thirdout")(third_layer)

fourth_layer = Dense(Ytrain.shape[1],activation='softmax',name = "fourth")(third_dropout)


history = Model(input_layer,fourth_layer)
history.compile(optimizer = "adam",loss="categorical_crossentropy",metrics=["accuracy"])

print (history.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 2559)]            0         
_________________________________________________________________
first (Dense)                (None, 1000)              2560000   
_________________________________________________________________
firstdout (Dropout)          (None, 1000)              0         
_________________________________________________________________
second (Dense)               (None, 800)               800800    
_________________________________________________________________
third (Dense)                (None, 1000)              801000    
_________________________________________________________________
thirdout (Dropout)           (None, 1000)              0         
_________________________________________________________________
fourth (Dense)               (None, 2559)              256155

In [14]:
# Model Training
history.fit(Xtrain, Ytrain, batch_size=BATCH_SIZE,epochs=NUM_EPOCHS, verbose=1,validation_split = 0.2)


Epoch 1/100
113/113 [==============================] - 5s 37ms/step - loss: 6.7311 - accuracy: 0.0502 - val_loss: 6.0928 - val_accuracy: 0.0663
Epoch 2/100
113/113 [==============================] - 4s 32ms/step - loss: 5.8936 - accuracy: 0.0682 - val_loss: 6.0444 - val_accuracy: 0.0747
Epoch 3/100
113/113 [==============================] - 4s 33ms/step - loss: 5.6907 - accuracy: 0.0761 - val_loss: 6.0318 - val_accuracy: 0.0858
Epoch 4/100
113/113 [==============================] - 4s 33ms/step - loss: 5.4255 - accuracy: 0.1077 - val_loss: 5.9674 - val_accuracy: 0.0997
Epoch 5/100
113/113 [==============================] - 4s 33ms/step - loss: 5.1682 - accuracy: 0.1320 - val_loss: 6.0025 - val_accuracy: 0.1053
Epoch 6/100
113/113 [==============================] - 4s 35ms/step - loss: 4.9352 - accuracy: 0.1504 - val_loss: 6.0884 - val_accuracy: 0.1050
Epoch 7/100
113/113 [==============================] - 4s 34ms/step - loss: 4.6779 - accuracy: 0.1726 - val_loss: 6.2218 - val_accuracy:

In [15]:

# Model Prediction
Y_pred = history.predict(Xtest)


# Sample check on Test data
print ("Prior bigram words","|Actual","|Predicted","\n")

for i in range(10):
    print (i,xtest_tg[i],"|",rev_dictnry[np.argmax(Ytest[i])],"|",rev_dictnry[np.argmax(Y_pred[i])])


Prior bigram words |Actual |Predicted 

0 the evening | beautiful | beautiful
1 slipped in | like | the
2 alice swallowing | down | her
3 an encouraging | tone | alice
4 waistcoat pocket | or | was
5 she went | on | on
6 that she | knew | was
7 down on | her | one
8 dormouse went | on | on
9 soup soup | of | and


In [16]:

import random

NUM_DISPLAY = 10
   
for i in random.sample(range(len(xtest_tg)),NUM_DISPLAY):
    print (i,xtest_tg[i],"|",rev_dictnry[np.argmax(Ytest[i])],"|",rev_dictnry[np.argmax(Y_pred[i])])

5479 how am | to | to
7434 that all | wrong | thank
1537 but what | happens | am
4837 who are | you | these
2546 more whatever | happens | at
1904 of hers | that | are
6910 way of | expecting | expressing
4641 you know | so | the
3855 she carried | the | it
4034 down but | generally | the
